In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# import missingno as msno  ##pip install missingno

pd.set_option('display.max_rows', 500) # your numbers here
pd.set_option('display.max_columns', 500)


### **Read File**

In [1]:
null_df = pd.read_csv('Data/car_v2.csv')

NameError: name 'pd' is not defined

### **Initial count of Rows and Column**

In [ ]:
null_df.shape

### **Are there duplicate rows?**

'Fed RSSD' uniquely identifies each bank.  There are no duplicate banks.

In [ ]:
null_df['Fed RSSD'].nunique()

In [ ]:
null_df.duplicated().head()

It appears that there are duplicates in columns 'namehcr and 'Bank Name'. Below is more info:

In [ ]:
null_df['namehcr'].value_counts().head(25)

In [ ]:
null_df['Bank Name'].value_counts().head(25)

In [ ]:
null_df([null_df['Fed RSSD'].value_counts()>1)

In [ ]:
null_df[null_df.duplicated('Fed RSSD')]

In [ ]:
null_df[null_df['namehcr']== 'WINTRUST FINANCIAL CORPORATION'].head(5)

In [ ]:
#null_df[null_df.duplicated(['Bank Name'], keep=False)]  # this code will display all duplicates

In [ ]:
null_df[null_df['Bank Name']== 'First State Bank'].head(5)


### **Count of null at the Column level **

### There are 127 rows and some rows with null values. Many of these null rows have a count of only nine nulls.

In [ ]:
null_df.isnull().sum().count()

In [ ]:
#null_df.isnull().sum()
null_df.isnull().sum().sort_values(ascending=False).head(45)                                

Another alternative: 

Let's find out how many missing data do we have.

First, let's count the number of null values

In [ ]:
total = null_df.isnull().sum().sort_values(ascending=False)
# Then, let's calculate the percentage of missing data per feature
percent = (null_df.isnull().sum()/null_df.isnull().count()).sort_values(ascending=False)*100
# Finally, let's concatenate Total and Percent into another dataframe
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(30)


### **More Analysis**



### Find a pattern: Are there several columns with null values at the same index rows?  

###  Retreive the row number for the solitary row which has "Number Employees" as null

In [ ]:
null_columns=null_df.columns[null_df.isnull().any()] 
# Step one - Assign null_columns

In [ ]:
print(null_df[null_df["Number Employees"].isnull()][null_columns])

# Step two - 
#Are there other columns with null value at the same index than sample column "Number Employees"
#The answer is Yes. It appears to be several colums with nine null value at the same index as seen in the results below.

### Removing null values at indexes = 268, 282, 313, 390, 3127, 3194, 4175, 4176, 4380

In [ ]:
#Testing null value at index row= 282
null_df.loc[282]['Number Employees']

In [ ]:
null_df.shape

In [ ]:
filtered_null_step1= null_df[null_df['Number Employees'].notnull()]

In [ ]:
filtered_null_step1.shape
# AS seenm, we removed a total of nine rows. Our dataset shape is now 5542 rowns (previously 5551 rows)

### We have removed most of the columns with null value count = 9 and 10.

As seen in code below, we still have several null values

In [ ]:
filtered_null_step1.isnull().sum().sort_values(ascending=False).head(21)

### In this step, we will drop those columns containing more than 80% of null values

In [ ]:
null_df = filtered_null_step1.columns[filtered_null_step1.isna().any()]

In [ ]:
#Columns containing nulls - At this point, we just visualizing all columns with null values
null_df

### Dropping Columns containing >80% of nulls

filtered_null_step2 = filtered_null_step1.dropna(thresh=500, axis=1)  #Keep only the rows with at least 500 non-na values

In [ ]:
filtered_null_step3 = filtered_null_step1.dropna(thresh=0.8*len(filtered_null_step1), axis=1)

In [ ]:
filtered_null_step2 = filtered_null_step1.dropna(thresh=0.8*len(filtered_null_step1), axis=1)

In [ ]:
#filtered_null_step2 = filtered_null_step1.loc[:, filtered_null_step1.isnull().sum() < 0.8*filtered_null_step1.shape[0]]

We have dropped ccolumns accountting with 80% of null values.

Let's see how many columns with null values we still have:

In [ ]:
filtered_null_step2.shape

In [ ]:
filtered_null_step2.isnull().sum().sort_values(ascending=False).head(16)

In [ ]:
#This is a visualization matrix very interesarting  from library "missingno" but it doesn't really work well with a large datasets.

#msno.matrix(filtered_null_step2)

### Fill out Columns containing nulls > 500 with its mean value

In [ ]:
filtered_null_step2["Credit Loss Prov to Chargeoffs"].describe()

In [ ]:
filtered_null_step2["Loan Loss Allow to noncurr Loans"].describe()

In [ ]:
filtered_null_step2.shape
# As seen, we removed a total of five columns.

### We are going to segment the banks by 'Total Assets' so that we may replace nulls with segment mean.

In [ ]:
#Small <$50M, Medium = $50M to $50B, Large = $50B to $3T
filtered_null_step2['Bank Size'] = pd.cut(filtered_null_step2['Total Assets'], [0,50000, 50000000, 3000000000], 
labels=['Small', 'Medium', 'Large'])

Step 1: Create a list of columns that are type float or int.  We will iterate over these columns and replace nulls 
with the mean value for the column based on banks segments small, medium and large.

Step 2: create a mask for banks that have null values in 'col_name' and are bank size
    
Step 3: get mean value for the col_name based on bank size.

Step 4: replace nulls with mean value for bank size segment

In [ ]:
columns = [col_name for col_name, dtype in filtered_null_step2.dtypes.iteritems() if (dtype == np.float or dtype == np.int)]

In [ ]:
for col_name in columns:
    if filtered_null_step2[col_name].isnull().any():
        #Replace nulls for small banks
        mask = (filtered_null_step2[col_name].isnull()) & (filtered_null_step2['Bank Size'] == 'Small')
        segment_mean = filtered_null_step2.loc[filtered_null_step2['Bank Size'] == 'Small', col_name].mean()
        filtered_null_step2.loc[mask, [col_name]] = segment_mean
        
        #Replace nulls for medium banks
        mask = (filtered_null_step2[col_name].isnull()) & (filtered_null_step2['Bank Size'] == 'Medium')
        segment_mean = filtered_null_step2.loc[filtered_null_step2['Bank Size'] == 'Medium', col_name].mean()
        filtered_null_step2.loc[mask, [col_name]] = segment_mean
        
        #Replace nulls for large banks
        mask = (filtered_null_step2[col_name].isnull()) & (filtered_null_step2['Bank Size'] == 'Large')
        segment_mean = filtered_null_step2.loc[filtered_null_step2['Bank Size'] == 'Large', col_name].mean()
        filtered_null_step2.loc[mask, [col_name]] = segment_mean

In [ ]:
filtered_null_step2.isnull().sum().sort_values(ascending=False).head(5)

In [ ]:
filtered_null_step2.to_csv('Data/car_v3.csv')

# Gonzalo's code for the last piece 
# What are we going to do with this code as far as presentation purposes are concerned?

1- First, I reassigned our daaset to a different name so that i can test my code.

In [ ]:
gon_df = filtered_null_step3

2- We set our segments using the "cut method" into 3 bins as seen in code below

In [ ]:
#Small <$50M, Medium = $50M to $50B, Large = $50B to $3T
gon_df['Bank Size'] = pd.cut(gon_df['Total Assets'], [0,50000, 50000000, 3000000000], 
labels=['Small', 'Medium', 'Large'])

3- Create a list of columns that are type float or int.  We will iterate over these columns and replace nulls 
with the mean value for the column based on banks segments small, medium and large.

In [ ]:
columns = [col_name for col_name, dtype in gon_df.dtypes.iteritems() if (dtype == np.float or dtype == np.int)]

Splitting the dasaset in three pieces

In [ ]:
sm_df = gon_df.loc[gon_df['Bank Size'] == 'Small']
md_df = gon_df.loc[gon_df['Bank Size'] == 'Medium']
lg_df = gon_df.loc[gon_df['Bank Size'] == 'Large']

Let's check their shape

In [ ]:
sm_df.shape

In [ ]:
md_df.shape

In [ ]:
lg_df.shape

Now, let's replace any null value with its average accordingly

In [ ]:
for col_name in columns:
    sm_df[col_name].fillna(sm_df[col_name].mean(), inplace = True)
    md_df[col_name].fillna(md_df[col_name].mean(), inplace = True)
    lg_df[col_name].fillna(lg_df[col_name].mean(), inplace = True)

Let's check if there still any null value left

In [ ]:
sm_df.loc[sm_df['Bank Size'] == 'Small'].isnull().sum().sort_values(ascending=False).head()

In [ ]:
md_df.isnull().sum().sort_values(ascending=False).head(5)

In [ ]:
#lg_df.isnull().sum().sort_values(ascending=False).head(5)

In this part, we will concatenate the three Dataframes together into one Dataframe called gon_df_conc and finally, we will create a cvs file

In [ ]:
frames = [df1, df2, df3]

In [5]: result = pd.concat(frames)

In [ ]:
frames =[sm_df, md_df, lg_df]

In [ ]:
gon_df_conc = pd.concat(frames)

In [ ]:
gon_df_conc.shape

In [ ]:
#gon_df_conc.to_csv('Data/car_v4.csv')